In [ ]:
import asyncio
import os
import re
import json
from urllib.parse import urlparse, urljoin
from email.header import decode_header

!pip -q install playwright
from playwright.async_api import async_playwright, TimeoutError as PWTimeoutError

!playwright install
!playwright install-deps

Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,592 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,864 kB]
Fetched 12.5 MB in 2s (5,597 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not

In [ ]:
BASE = "https://enlinea.senace.gob.pe/BuscadorNormas/#/norma/{}"
OUT_DIR = "/content/drive/MyDrive/Maestria UNI/PLN/TrabajoFinal/senace_normas"
START_ID = 100
END_ID = 500

In [ ]:
def safe_filename(name: str) -> str:
    name = re.sub(r"[\\/*?:\"<>|]", "_", name)
    name = re.sub(r"\s+", " ", name).strip()
    return name[:180] if len(name) > 180 else name


def guess_ext_from_url(url: str) -> str:
    path = urlparse(url).path.lower()
    if path.endswith(".pdf"):
        return ".pdf"
    if path.endswith(".doc"):
        return ".doc"
    if path.endswith(".docx"):
        return ".docx"
    if path.endswith(".xls"):
        return ".xls"
    if path.endswith(".xlsx"):
        return ".xlsx"
    # SENACE /download suele devolver PDF sin extensión en la URL
    return ".pdf"


def decode_mime_word(s: str) -> str:
    """
    Decodifica nombres tipo =?utf-8?B?...?= o =_utf-8_B_..._=
    """
    try:
        # normaliza el formato raro con guiones bajos
        s = s.replace("=_utf-8_B_", "=?utf-8?B?").replace("_=", "?=")

        parts = decode_header(s)
        decoded = ""
        for text, charset in parts:
            if isinstance(text, bytes):
                decoded += text.decode(charset or "utf-8", errors="ignore")
            else:
                decoded += text
        return decoded
    except Exception:
        return s


def pick_filename_from_headers(headers: dict, fallback_name: str, url: str) -> str:
    """
    Obtiene el nombre real desde Content-Disposition.
    Si viene codificado, lo decodifica.
    Si no existe, usa fallback + extensión inferida.
    """
    cd = ""
    for k, v in headers.items():
        if k.lower() == "content-disposition":
            cd = v
            break

    if cd:
        # filename* (UTF-8 moderno RFC5987)
        m = re.search(r'filename\*=(?:UTF-8\'\')?([^;]+)', cd, flags=re.I)
        if m:
            name = decode_mime_word(m.group(1).strip('"'))
            return safe_filename(name)

        # filename clásico
        m = re.search(r'filename="?([^";]+)"?', cd, flags=re.I)
        if m:
            name_raw = m.group(1)
            name = decode_mime_word(name_raw)
            return safe_filename(name)

    # fallback
    ext = guess_ext_from_url(url)
    name = safe_filename(fallback_name) or "archivo"
    if not re.search(r"\.(pdf|docx?|xlsx?)$", name, flags=re.I):
        name += ext
    return name


async def download_file(context, file_url: str, dest_folder: str,
                        referer: str, link_text: str,
                        norma_id: int | None = None,
                        file_index: int | None = None):
    """
    Descarga el pdf
    """

    # Petición HTTP al endpoint /download
    resp = await context.request.get(
        file_url,
        timeout=90000,
        headers={"Referer": referer}
    )
    if not resp.ok:
        raise Exception(f"HTTP {resp.status} descargando {file_url}")

    # 1) Nombre base: texto del link
    base_name = (link_text or "").strip()
    if not base_name:
        base_name = "archivo"

    base_name = safe_filename(base_name)

    # 2) Asegurar extensión
    if not re.search(r"\.(pdf|docx?|xlsx?)$", base_name, flags=re.I):
        # Por URL
        ext = guess_ext_from_url(file_url)

        # Mejorar por Content-Type si es posible
        ct = (resp.headers.get("content-type") or "").lower()
        if "pdf" in ct:
            ext = ".pdf"
        elif "msword" in ct or "word" in ct:
            ext = ".doc"
        elif "spreadsheet" in ct or "excel" in ct:
            ext = ".xlsx"

        base_name += ext

    filename = base_name

    # 3) Prefijo opcional con ID e índice (para distinguir archivos)
    if norma_id is not None:
        name_body, ext = os.path.splitext(filename)
        if file_index is not None:
            filename = f"{norma_id}_{file_index}_{name_body}{ext}"
        else:
            filename = f"{norma_id}_{name_body}{ext}"

    dest = os.path.join(dest_folder, filename)

    # 4) Guardar bytes
    with open(dest, "wb") as wf:
        wf.write(await resp.body())

    return dest




async def click_tab_if_exists(page, tab_text: str):
    """
    Click en tab por texto si existe. No revienta si no está.
    """
    try:
        loc = page.locator("ul.nav.nav-tabs").locator(f"text={tab_text}")
        if await loc.count() > 0:
            await loc.first.click(timeout=6000)
            await page.wait_for_timeout(800)
    except Exception:
        pass


async def robust_wait_norma_loaded(page, timeout_ms: int = 90000):
    """
    Espera robusta para Angular SPA:
    - Espera tabs
    - Espera que haya texto real en algún bloque .norma-info-item .body
    - Espera que body tenga suficiente texto visible
    """
    await page.wait_for_selector("ul.nav.nav-tabs", timeout=timeout_ms)

    await page.wait_for_function(
        """
        () => {
            const bodies = Array.from(document.querySelectorAll('.norma-info-item .body'));
            return bodies.length > 0 && bodies.some(b => (b.innerText || '').trim().length > 5);
        }
        """,
        timeout=timeout_ms
    )

    await page.wait_for_function(
        """
        () => {
            const t = (document.body?.innerText || '').trim();
            return t.length > 80;
        }
        """,
        timeout=timeout_ms
    )


async def extract_meta(page) -> dict:
    """
    Extrae metadatos con selectores mejorados para capturar 'Temas'
    y filtrar elementos irrelevantes (Exportar/Compartir).
    """
    #await page.wait_for_selector('[class^="norma-info-items"]', timeout=12000)
    await page.wait_for_selector('.norma-info-item', timeout=20000)


    return await page.evaluate("""
    () => {
      const out = {};
      // Buscamos todos los bloques de información
      const items = document.querySelectorAll('.norma-info-item .body');

      items.forEach(body => {
        const labelEl = body.querySelector('label.title');
        if (!labelEl) return;

        const label = labelEl.innerText.replace(':', '').trim();

        // Intentamos obtener el texto de .info-txt
        // o de cualquier div/span que siga al label (para casos especiales de Angular)
        let valueEl = body.querySelector('.info-txt');

        // Si no existe .info-txt, buscamos el siguiente elemento hermano del label
        if (!valueEl) {
            valueEl = labelEl.nextElementSibling;
        }

        if (valueEl) {
            let text = valueEl.innerText.trim();

            // Si el texto está vacío pero hay una lista (como en Sugerencias),
            // no lo agregamos, o lo procesamos distinto.
            // Para 'Temas', esto asegura capturar el texto plano.
            if (label && text.length > 0) {
                out[label] = text;
            }
        }
      });
      return out;
    }
    """)

async def extract_header_optional(page) -> tuple[str, str]:
    """
    Extrae lbltituloNorma / lblNorma si existen (sin bloquear).
    """
    titulo_norma = ""
    codigo_norma = ""

    lbl1 = page.locator("#lbltituloNorma")
    if await lbl1.count() > 0:
        titulo_norma = (await lbl1.first.inner_text()).strip()

    lbl2 = page.locator("#lblNorma")
    if await lbl2.count() > 0:
        codigo_norma = (await lbl2.first.inner_text()).strip()

    return titulo_norma, codigo_norma


async def extract_texto_completo(page) -> str:
    """
    Intenta extraer el texto completo:
    - preferir nodos [id^="article"] (cuando existen)
    - fallback: texto general del body
    """
    # Asegurar tab "Texto completo" si existe
    await click_tab_if_exists(page, "Texto completo")

    # 1) Extraer H1 (ordenados como aparecen en el DOM)
    h1_texts = await page.eval_on_selector_all(
        "h1",
        """
        els => els
          .map(e => (e.innerText || '').trim())
          .filter(t => t.length > 0)
        """
    )
    codigo_norma = h1_texts[0] if len(h1_texts) >= 1 else ""
    titulo_norma = h1_texts[1] if len(h1_texts) >= 2 else ""

    # 2) Extraer cuerpo del texto
    articles_text = ""
    try:
        # Algunos casos tardan en generar los article_*
        await page.wait_for_selector('[id^="norma-view"]', timeout=12000)
        articles_text = await page.eval_on_selector_all(
            '[id^="norma-view"]',
            """
            els => els
              .map(e => (e.innerText || '').trim())
              .filter(t => t.length > 0)
              .join('\\n\\n')
            """
        )
    except PWTimeoutError:
        articles_text = ""

    full_text = await page.evaluate("() => (document.body.innerText || '').trim()")
    # return articles_text if len(articles_text) >= 80 else full_text
    cuerpo = articles_text if len(articles_text) >= 80 else full_text

    return codigo_norma, titulo_norma, cuerpo


from urllib.parse import urljoin  # asegúrate de tener esto importado

async def download_archivos_tab(page, context, norma_folder: str,
                                referer_url: str, norma_id: int):

    try:
        tab_archivos = page.locator("ul.nav.nav-tabs >> text=Archivos")
        if await tab_archivos.count() > 0:
            await tab_archivos.first.scroll_into_view_if_needed()
            await tab_archivos.first.click(timeout=8000)
        else:
            # Fallback: texto "Archivos" en cualquier lado
            tab_archivos = page.locator("text=Archivos")
            if await tab_archivos.count() > 0:
                await tab_archivos.first.scroll_into_view_if_needed()
                await tab_archivos.first.click(timeout=8000)
    except Exception:
        pass  # si no se puede clickear igual seguimos

    await page.wait_for_timeout(1500)

    # 2) Buscar TODOS los links de descarga en la página
    archivos = await page.eval_on_selector_all(
        "a[href*='/NormasAmbientales/api/documentos/'][href*='/download']",
        """
        els => els.map((a, i) => ({
          href: a.href,
          text: (a.innerText || a.textContent || '').trim(),
          idx: i + 1
        })).filter(x => x.href)
        """
    )

    if not archivos:
        print("  -> No se encontraron archivos de descarga (ningún href con /NormasAmbientales/api/documentos/.../download)")
        return

    # Quitar duplicados
    seen = set()
    archivos_clean = []
    for a in archivos:
        if a["href"] not in seen:
            seen.add(a["href"])
            archivos_clean.append(a)

    files_folder = os.path.join(norma_folder, "archivos")
    os.makedirs(files_folder, exist_ok=True)

    print(f"  -> Archivos encontrados: {len(archivos_clean)}")
    # DEBUG: ver textos reales de los links
    for a in archivos_clean:
        print(f"     Link: {a['href']}  |  Texto: {a['text']}")

    # 3) Descargar
    for idx, a in enumerate(archivos_clean, start=1):
        href = a["href"]
        text = a["text"] or f"archivo_{idx}"

        absolute_url = urljoin(referer_url, href)

        try:
            saved = await download_file(
                context=context,
                file_url=absolute_url,
                dest_folder=files_folder,
                referer=referer_url,
                link_text=text,
                norma_id=norma_id,
                file_index=idx
            )
            print(f"     Descargado: {os.path.basename(saved)}")
        except Exception as e:
            print(f"     Error descargando {absolute_url}: {e}")


In [ ]:
async def main():
    os.makedirs(OUT_DIR, exist_ok=True)

    # 1. Initialize an empty list
    all_normas_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=["--disable-blink-features=AutomationControlled"],
        )

        context = await browser.new_context(
            locale="es-PE",
            user_agent=(
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/120.0.0.0 Safari/537.36"
            ),
            viewport={"width": 1400, "height": 900},
        )

        async def block_heavy(route):
            if route.request.resource_type in ("image", "font"):
                await route.abort()
            else:
                await route.continue_()

        await context.route("**/*", block_heavy)

        for norma_id in range(START_ID, END_ID + 1):
            url = BASE.format(norma_id)
            norma_folder = os.path.join(OUT_DIR, str(norma_id))
            os.makedirs(norma_folder, exist_ok=True)

            print(f"\n[{norma_id}] {url}")

            page = await context.new_page()
            try:

                await page.goto(url, wait_until="domcontentloaded", timeout=90000)

                await robust_wait_norma_loaded(page, timeout_ms=90000)


                # titulo_norma, codigo_norma = await extract_header_optional(page)



                # Meta
                meta = await extract_meta(page)
                fecha_publicacion = meta.get("Fecha de publicación", "")
                organismo = meta.get("Organismo", "")
                estado = meta.get("Estado", "")
                temas = meta.get("Temas", "")

                # Texto completo
                # cuerpo_texto = await extract_texto_completo(page)
                codigo_h1, titulo_h1, cuerpo_texto = await extract_texto_completo(page)
                if not cuerpo_texto or len(cuerpo_texto) < 80:
                    print("  -> No se encontró texto suficiente")
                    await page.close()
                    continue

                # 3a. Create dictionary and populate it with extracted metadata
                norma_data = {
                    "id": norma_id,
                    "titulo": titulo_h1,
                    "codigo": codigo_h1,
                    "fecha_publicacion": fecha_publicacion,
                    "organismo": organismo,
                    "estado": estado,
                    "temas": temas
                }

                # Guardar TXT
                final_text = (
                    f"TÍTULO:\n{titulo_h1}\n\n"
                    f"NORMA:\n{codigo_h1}\n\n"
                    f"FECHA DE PUBLICACIÓN:\n{fecha_publicacion}\n\n"
                    f"ORGANISMO:\n{organismo}\n\n"
                    f"ESTADO:\n{estado}\n\n"
                    f"TEMAS:\n{temas}\n\n"
                    f"{'-' * 60}\n\n"
                    f"{cuerpo_texto}\n"
                )

                txt_path = os.path.join(norma_folder, f"norma_{norma_id}.txt")
                with open(txt_path, "w", encoding="utf-8") as f:
                    f.write(final_text)

                # Guardar JSON
                json_path = os.path.join(norma_folder, f"norma_{norma_id}.json")
                with open(json_path, "w", encoding="utf-8") as jf:
                    json.dump({
                        "id": norma_id,
                        "titulo": titulo_h1,
                        "codigo": codigo_h1,
                        "fecha_publicacion": fecha_publicacion,
                        "organismo": organismo,
                        "estado": estado,
                        "temas": temas,
                        "url": url
                    }, jf, ensure_ascii=False, indent=2)

                print("  -> TXT y JSON guardados")

                all_normas_data.append(norma_data)

                await download_archivos_tab(
                    page,
                    context,
                    norma_folder,
                    referer_url=url,
                    norma_id=norma_id,
                )

            except PWTimeoutError as e:
                print(f"  !! Timeout cargando la norma: {e}")
            except Exception as e:
                print(f"  !! Error: {e}")
            finally:
                await page.close()

        await browser.close()

if __name__ == "__main__":
    await main()


[121] https://enlinea.senace.gob.pe/BuscadorNormas/#/norma/121
  -> TXT y JSON guardados
  -> No se encontraron archivos de descarga (ningún href con /NormasAmbientales/api/documentos/.../download)

[122] https://enlinea.senace.gob.pe/BuscadorNormas/#/norma/122
  -> TXT y JSON guardados
  -> Archivos encontrados: 1
     Link: https://enlinea.senace.gob.pe/NormasAmbientales/api/documentos/97/download  |  Texto: EXP-DS-002-2021-MINAM EXPOSICION DE MOTIVOS.pdf
     Descargado: 122_1_EXP-DS-002-2021-MINAM EXPOSICION DE MOTIVOS.pdf

[123] https://enlinea.senace.gob.pe/BuscadorNormas/#/norma/123
  !! Timeout cargando la norma: Page.wait_for_function: Timeout 90000ms exceeded.

[124] https://enlinea.senace.gob.pe/BuscadorNormas/#/norma/124
  -> TXT y JSON guardados
  -> No se encontraron archivos de descarga (ningún href con /NormasAmbientales/api/documentos/.../download)

[125] https://enlinea.senace.gob.pe/BuscadorNormas/#/norma/125
  -> TXT y JSON guardados
  -> Archivos encontrados: 1
  